In [2]:
import glob
import os
import re
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json

In [5]:
paths = glob.glob('../results_hyper/*.txt')

In [6]:
path = paths[0]

In [28]:
def extract_row(path):
    with open(path) as fp:
        lines = fp.readlines()

        sel = list(filter(lambda line: line.startswith('{"startupTime":'), lines))
        row = json.loads(sel[0])

        _, mode, _, run = os.path.basename(path).split('-')
        run = int(run.replace('.txt', ''))
        row['mode'] = mode
        row['run'] = run
        row['path'] = path

        sel = list(filter(lambda line: 'LAMBDA compute took' in line, lines))[0]
        cost = float(sel[sel.find('cost $')+6:].strip())

        row['cost'] = cost

        return row

In [30]:
rows = []
for path in paths:
    rows.append(extract_row(path))
    
df = pd.DataFrame(rows)

In [34]:
df.groupby('mode').agg(['mean', 'std']).reset_index()

mode startupTime              jobTime            run            cost  \
                  mean       std       mean       std mean      std   mean   
0    hyper    0.208996  0.018164  47.310272  1.614773  5.5  3.02765  0.592   
1  nohyper    0.233714  0.019562  63.819428  0.873001  5.5  3.02765  0.932   

             
        std  
0  0.004216  
1  0.004216

In [69]:
#speedup
63.819428 / 47.310272

1.348955000723733

In [71]:
1.0 - 0.592 / 0.932 

0.36480686695278974

In [35]:
# maybe use different sampling modes?

In [36]:
# generate the table from path info

In [53]:
def extract_path_stats_row(path):
    with open(path) as fp:
        lines = fp.readlines()
        needle = 'LAMBDA paths rows took:'
        sel = list(filter(lambda line: needle in line, lines))[0]

        sel = sel[sel.find('normal:'):]

        _, normal, _, general, _, interpreter, _, unresolved = sel.strip().split(' ')


        row = {'normal' : int(normal),
               'general': int(general),
               'interpreter' : int(interpreter),
               'unresolved' : int(unresolved)}

        _, mode, _, run = os.path.basename(path).split('-')
        row['mode'] = mode
    return row

In [54]:
df_rows = pd.DataFrame([extract_path_stats_row('../results_hyper/flights-hyper-run-1.txt'),
                       extract_path_stats_row('../results_hyper/flights-nohyper-run-1.txt')])

In [68]:
df_rows.style.format(thousands=",")
print(df_rows[['mode', 'normal', 'general', 'interpreter', 'unresolved']].to_latex(index=None))

\begin{tabular}{lrrrr}
\toprule
    mode &     normal &    general &  interpreter &  unresolved \\
\midrule
   hyper &  149930703 &   44830574 &      7687788 &      238590 \\
 nohyper &     448620 &  179364496 &     22635949 &      238590 \\
\bottomrule
\end{tabular}



,normal,general,interpreter,unresolved,mode
0,"149,930,703","44,830,574","7,687,788","238,590",hyper
1,"448,620","179,364,496","22,635,949","238,590",nohyper


In [73]:
!cat ../results_hyper/flights-hyper-run-1.txt

[2022-06-14 08:47:14.544] [memory] [info] allocated bitmap managed memory region (2.00 GB, 32.00 MB block size)
[2022-06-14 08:47:14.544] [aws-lambda] [info] Execution over lambda with 10000MB
[2022-06-14 08:47:14.544] [aws-lambda] [info] Adjusted lambda size to 10000MB
[2022-06-14 08:47:14.686] [aws-lambda] [info] Found AWS Lambda function tuplex-lambda-runner (10000MB)
[2022-06-14 08:47:14.686] [aws-lambda] [info] Using Lambda running on x86_64
[2022-06-14 08:47:14.686] [aws-lambda] [info] Lambda client uses JSON AST serialization format.
[2022-06-14 08:47:14.963] [fileinputoperator] [info] found 410 files (83.51 GB) to process.
[2022-06-14 08:47:15.035] [global] [info] sampled s3://tuplex-public/data/flights_all/flights_on_time_performance_1987_10.csv on 256.00 KB
[2022-06-14 08:47:15.035] [fileinputoperator] [info] Parallel sample fetch done.
[2022-06-14 08:47:15.035] [fileinputoperator] [info] Filling sample cache for csv operator took 0.071980s (1 entries, 0 rows)
[2022-06-14 08: